In [ ]:
import strax 
import straxen 
import numpy as np
import matplotlib.pyplot as plt
import cutax
import pandas as pd

In [ ]:
st = cutax.contexts.xenonnt_v8()

In [ ]:
help(st.select_runs)

In [ ]:
st.select_runs(run_mode = 'tpc_ar37', include_tags = ['kr83m_and_ar37'], exclude_tags = ['messy', 'lt_3d_after_ramp'])

In [ ]:
cal_run_id = '034373'

In [ ]:
kr_peaks = strax.(run_id='034373', targets='peaks', progress_bar=True)

In [ ]:
kr_pb = st.get_array(run_id='034373', targets='peak_basics')

In [ ]:
kr_peaks = st.get_array(run_id='034373', targets='peaks')

In [ ]:
kr_apc = st.get_array(run_id='034373', targets='event_area_per_channel')

In [ ]:
len(kr_apc)

In [ ]:
len(kr_peaks)

In [ ]:
kr_peak_pos = st.get_array(run_id='034373', targets='peak_positions_mlp')

In [ ]:
kr_peak_pos

In [ ]:
a = 44
def addnum(a):
    b = a + 25
    return b

In [ ]:
check = addnum

In [ ]:
check(23)

In [ ]:
f = addnum(a)
f

In [ ]:
len(f)

In [ ]:
kr_peaks.dtype

In [ ]:
peaks_and_pos = np.append(kr_peaks, kr_peak_pos)

In [ ]:
len(kr_peaks)

In [ ]:
len(kr_peak_pos)

In [ ]:
kr_eb = st.get_array(run_id='034373', targets='event_basics')

In [ ]:
kr_eb.dtype

In [ ]:
len(kr_eb)

In [ ]:
kr_eb.dtype

In [ ]:
kr_eb['s2_x']

In [ ]:
len(kr_peaks)

In [ ]:
kr_peaks.dtype

In [ ]:
help(st.make)

In [ ]:
len(kr_peaks)

In [ ]:
st.register_all(cutax.cuts)
kr_events = st.get_df(cal_run_id, targets =('events', 'cut_Kr_SingleS1S2'))

In [ ]:
kr_events

In [ ]:
kr_events_all = st.get_df(cal_run_id, targets =('event_basics'))

In [ ]:
kr_events_all

In [ ]:
def touching_windowsv2(things, containers, window=0):
    """Return array of (start, exclusive end) indices into things which extend
    to within window of the container, for each container in containers.
    :param things: Sorted array of interval-like data
    :param containers: Sorted array of interval-like data
    :param window: threshold distance for touching check
    For example:
       - window = 0: things must overlap one sample
       - window = -1: things can start right after container ends
         (i.e. container endtime equals the thing starttime, since strax
          endtimes are exclusive)
    """
    return _touching_windows(
        things['time'], things['endtime'],
        containers['s1_time'], containers['s2_endtime'],#(containers),
        window=window)


#@numba.njit(nogil=True, cache=True)
def _touching_windows(thing_start, thing_end,
                      container_start, container_end,
                      window=0):
    result = np.zeros((len(container_start), 2), dtype=np.int32)
    n = len(thing_start)
    left_i = right_i = 0

    for i, t0 in enumerate(container_start):
        t1 = container_end[i]

        while left_i <= n - 1 and thing_end[left_i] <= t0 - window:
            # left_i ends before the window starts (so it's still outside)
            left_i += 1
        # Now left_i is the first index inside the window
        # -- unless it is outside the array, in which case right_i
        # will also be.

        while right_i <= n - 1 and thing_start[right_i] < t1 + window:
            # right_i starts before the window ends (so it could be inside)
            right_i += 1
        # Now right_i is the last index inside the window
        # or outside the array.

        result[i] = left_i, right_i

    return result

In [ ]:
def touch_window(thing_start, 
                 container_start, container_end,
                 window=0):
    result = np.zeros((len(container_start), 2), dtype=np.int32)
    n = len(thing_start)
    left_i = right_i = 0

    for i, t0 in enumerate(container_start):
        t1 = container_end[i]
        
        #ignore data as long as the endtime of the peaklets is less than the start time of the container
        while left_i <= n - 1 and thing_end[left_i] <= t0 - window:
            # left_i ends before the window starts (so it's still outside)
            left_i += 1
        # Now left_i is the first index inside the window
        # -- unless it is outside the array, in which case right_i
        # will also be.

        while right_i <= n - 1 and thing_start[right_i] < t1 + window:
            # right_i starts before the window ends (so it could be inside)
            right_i += 1
        # Now right_i is the last index inside the window
        # or outside the array.

        result[i] = left_i, right_i

    return result

In [ ]:
def get_data_in_time_window(peaklets_start, peak_window_start, peak_window_end, time_resets):
    '''
    The purpose of this function is to return the indecies of the peaklets which fall within the time
    range of two peaks, the qualification for this is that the peaklets start time is within the desired
    time frame (possibly questionable assumption, specially for S2s)
    '''
    result = np.zeros((len(peak_window_start), 2), dtype=np.int32)
    n = len(peaklets_start)
    time_index = endtime_index = 0 #indecies of peaklets to check
    time_check = 0 #index to reset the time when needed
    

    
    for i, t0 in enumerate(peak_window_start):
        if time_resets[time_check] == i:
            time_index = endtime_index = 0 #reset the timer once in a while since that data is not all organizing according to time
            time_check += 1
            print("Reset number " + str(time_check))
            
        #gap_start = 0
        #gap_end = 0
        while peaklets_start[time_index] < t0:
            time_index += 1
            
        #gap_start = i
        endtime_index = time_index
        while peaklets_start[endtime_index] < peak_window_end[i]:
            endtime_index += 1
            if endtime_index == len(peaklets_start):
                break
            
        result[i] = time_index, endtime_index
        
    return result
        

In [ ]:
help(st.make)

In [ ]:
kr_events_all.s1_time - kr_events_all.time

In [ ]:
s1t_t = kr_events_all.s1_time - kr_events_all.time

In [ ]:
np.where(s1t_t == np.min(s1t_t))

In [ ]:
np.min(s2t_t[1:-10])

In [ ]:
s2t_t[20:30]

In [ ]:
np.min(s2t_t[1:])

In [ ]:
np.max(s2t_t[1:])

In [ ]:
s2et_et = kr_events_all.endtime - kr_events_all.s2_endtime

In [ ]:
s2et_et

In [ ]:
np.max(s2et_et)

In [ ]:
np.min(s2et_et)

In [ ]:
kr_events_array = st.get_array(cal_run_id, targets =('events', 'cut_Kr_SingleS1S2', 'cut_Kr_DoubleS1_SingleS2'))

In [ ]:
print(cutax.cut_versions(st).to_string(index=False))

In [ ]:
kr_events_array

In [ ]:
kr_peaklets_array = st.get_array(cal_run_id, targets =('peaklets'))

In [ ]:
kr_pb_array = st.get_array(cal_run_id, targets =('peak_basics'))

In [ ]:
kr_eb_array = st.get_array(cal_run_id, targets =('event_basics'))

In [ ]:
kr_eb_df = st.get_df(cal_run_id, targets =('event_basics'))

In [ ]:
kr_peaklets = st.get_array(cal_run_id, targets =('peaklets'))

In [ ]:
len(kr_peaklets)

In [ ]:
len(kr_peaks)

In [ ]:
kr_peaklets.dtype

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(kr_peaklets['area'], kr_peaklets['rise_time'], s=0.1, color = 'blue', alpha = 0.5)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
np.unique(kr_peaklets['type'])

In [ ]:
kr_eb_array['time']

In [ ]:
kr_eb_array['endtime']

In [ ]:
kr_eb_array['s1_time']

In [ ]:
kr_eb_array['s2_endtime']

In [ ]:
kr_eb_array['endtime'] - kr_eb_array['s2_endtime']

In [ ]:
event_t_minus_s1t = kr_eb_array['time'] - kr_eb_array['s1_time']

In [ ]:
event_t_minus_s1t

In [ ]:
np.min(event_t_minus_s1t[1:])

In [ ]:
peak_pro = st.get_array(cal_run_id,'peak_proximity')

In [ ]:
kr_events_array[kr_events_array['cut_Kr_SingleS1S2'] == True]

In [ ]:
kr_events_array[1:10]

In [ ]:
kr_eb_array['s1_time']

In [ ]:
kr_eb_array['s1_time'][20:30]

In [ ]:
g = kr_eb_array[kr_events_array['cut_Kr_SingleS1S2'] == True]

In [ ]:
w2 = touching_windowsv2(kr_pb_array, g)

In [ ]:
s2= kr_pb_array[kr_pb_array['type']== 2]

In [ ]:
s2s = touching_windowsv2(s2, kr_eb_array[kr_events_array['cut_Kr_SingleS1S2'] == True])

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(s2['area'][s2s], s2['rise_time'][s2s], s=0.1, color = 'blue', alpha = 0.5)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
cal_run_id = '034373'
kr_events_array = st.get_array(cal_run_id, targets =('events', 'cut_Kr_SingleS1S2', 'cut_Kr_DoubleS1_SingleS2'))
kr_pb_array = st.get_array(cal_run_id, targets =('peak_basics'))
window1 = strax.touching_windows(kr_pb_array, kr_events_array[kr_events_array['cut_Kr_SingleS1S2'] == True])
store_values = []
for i in np.arange(len(window1)):
    store_values.append(np.arange(int(window1[i,0]), window1[i,1]+1))
    
values = np.concatenate(store_values)
fig = plt.figure(figsize=(12,6))
plt.scatter(kr_pb_array['area'][values], kr_pb_array['rise_time'][values], s=0.1, color = 'blue', alpha = 0.5)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
kr_cleaning = kr_pb_array[values]
kr_s1 = kr_cleaning[kr_cleaning['type'] == 1]
kr_s2 = kr_cleaning[kr_cleaning['type'] == 2]

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(kr_s1['area'], kr_s1['rise_time'], s=0.1, color = 'blue', alpha = 0.5)
plt.scatter(kr_s2['area'], kr_s2['rise_time'], s=0.1, color = 'green', alpha = 0.5)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(kr_s1[kr_s1['area'] > 100]['area'], kr_s1[kr_s1['area'] > 100]['rise_time'], s=0.1, color = 'blue', alpha = 0.5)
plt.scatter(kr_s2[kr_s2['area'] > 100]['area'], kr_s2[kr_s2['area'] > 100]['rise_time'], s=0.1, color = 'green', alpha = 0.5)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
np.append(kr_s1, )

In [ ]:
cal_run_id = '034373'
kr_events_array2 = st.get_array(cal_run_id, targets =('events', 'cut_Kr_SingleS1S2', 'cut_Kr_DoubleS1_SingleS2'))
kr_pb_array2 = st.get_array(cal_run_id, targets =('peak_basics'))
window2 = strax.touching_windows(kr_pb_array2, kr_events_array2[kr_events_array2['cut_Kr_DoubleS1_SingleS2'] == True])
store_values2 = []
for i in np.arange(len(window2)):
    store_values2.append(np.arange(int(window2[i,0]), window2[i,1]))
    
values2 = np.concatenate(store_values2)
fig = plt.figure(figsize=(12,6))
plt.scatter(kr_pb_df['area'][values2], kr_pb_df['rise_time'][values2], s=0.1, color = 'blue', alpha = 0.5)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
values2

In [ ]:
kr_singleS1S2_df = kr_pb_df.iloc[values]

In [ ]:
kr_singleS1S2_df_s1 = kr_singleS1S2_df[kr_singleS1S2_df['area'] > 200]
kr_singleS1S2_df_s1 = kr_singleS1S2_df_s1[kr_singleS1S2_df_s1['area'] < 1000]
kr_singleS1S2_df_s1 = kr_singleS1S2_df_s1[kr_singleS1S2_df_s1['rise_time'] > 30]
kr_singleS1S2_df_s1 = kr_singleS1S2_df_s1[kr_singleS1S2_df_s1['rise_time'] < 200]

In [ ]:
kr_singleS1S2_df_s2 = kr_singleS1S2_df[kr_singleS1S2_df['area'] > 3000]
kr_singleS1S2_df_s2 = kr_singleS1S2_df_s2[kr_singleS1S2_df_s2['area'] < 20000]
kr_singleS1S2_df_s2 = kr_singleS1S2_df_s2[kr_singleS1S2_df_s2['rise_time'] > 200]
kr_singleS1S2_df_s2 = kr_singleS1S2_df_s2[kr_singleS1S2_df_s2['rise_time'] < 20000]

In [ ]:
kr_dS1sS2_df = kr_pb_df.iloc[values2]

In [ ]:
kr_dS1sS2_df_s1 = kr_dS1sS2_df[kr_dS1sS2_df['area'] > 40]
kr_dS1sS2_df_s1 = kr_dS1sS2_df_s1[kr_dS1sS2_df_s1['area'] < 600]
kr_dS1sS2_df_s1 = kr_dS1sS2_df_s1[kr_dS1sS2_df_s1['rise_time'] > 20]
kr_dS1sS2_df_s1 = kr_dS1sS2_df_s1[kr_dS1sS2_df_s1['rise_time'] < 100]

In [ ]:
kr_dS1sS2_df_s2 = kr_dS1sS2_df[kr_dS1sS2_df['area'] > 3000]
#kr_dS1sS2_df_s2 = kr_dS1sS2_df_s2[kr_dS1sS2_df_s2['area'] < 600]
kr_dS1sS2_df_s2 = kr_dS1sS2_df_s2[kr_dS1sS2_df_s2['rise_time'] > 100]
#kr_dS1sS2_df_s2 = kr_dS1sS2_df_s2[kr_dS1sS2_df_s2['rise_time'] < 100]

In [ ]:
len(kr_singleS1S2_df_s1)

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(kr_singleS1S2_df_s1['area'], kr_singleS1S2_df_s1['rise_time'], s=0.1, color = 'blue', alpha = 0.5)
plt.scatter(kr_singleS1S2_df_s2['area'], kr_singleS1S2_df_s2['rise_time'], s=0.1, color = 'green', alpha = 0.5)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(kr_dS1sS2_df_s1['area'], kr_dS1sS2_df_s1['rise_time'], s=0.1, color = 'blue', alpha = 0.5)
plt.scatter(kr_dS1sS2_df_s2['area'], kr_dS1sS2_df_s2['rise_time'], s=0.1, color = 'green', alpha = 0.5)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
start_time_kr = np.zeros(len(kr_singleS1S2_df_s1)+len(kr_singleS1S2_df_s2)+len(kr_dS1sS2_df_s1)+len(kr_dS1sS2_df_s2), dtype = int)
endtime_kr = np.zeros(len(kr_singleS1S2_df_s1)+len(kr_singleS1S2_df_s2)+len(kr_dS1sS2_df_s1)+len(kr_dS1sS2_df_s2), dtype = int)

In [ ]:
#mark start time of krypton events starting with S1s
start_time_kr[:len(kr_singleS1S2_df_s1)] = kr_singleS1S2_df_s1['time']
start_time_kr[len(kr_singleS1S2_df_s1):len(kr_dS1sS2_df_s1)+len(kr_singleS1S2_df_s1)] = kr_dS1sS2_df_s1['time']
start_time_kr[len(kr_singleS1S2_df_s1)+len(kr_dS1sS2_df_s1):len(kr_singleS1S2_df_s1)+len(kr_dS1sS2_df_s1)+len(kr_singleS1S2_df_s2)] = kr_singleS1S2_df_s2['time']
start_time_kr[len(kr_singleS1S2_df_s1)+len(kr_dS1sS2_df_s1)+len(kr_singleS1S2_df_s2):] = kr_dS1sS2_df_s2['time']

In [ ]:
#mark endtime of krypton events starting with S1s
endtime_kr[:len(kr_singleS1S2_df_s1)] = kr_singleS1S2_df_s1['endtime']
endtime_kr[len(kr_singleS1S2_df_s1):len(kr_dS1sS2_df_s1)+len(kr_singleS1S2_df_s1)] = kr_dS1sS2_df_s1['endtime']
endtime_kr[len(kr_singleS1S2_df_s1)+len(kr_dS1sS2_df_s1):len(kr_singleS1S2_df_s1)+len(kr_dS1sS2_df_s1)+len(kr_singleS1S2_df_s2)] = kr_singleS1S2_df_s2['endtime']
endtime_kr[len(kr_singleS1S2_df_s1)+len(kr_dS1sS2_df_s1)+len(kr_singleS1S2_df_s2):] = kr_dS1sS2_df_s2['endtime']

Now that I have the krypton peaks by time lets get the overall data and their times with this

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(kr_singleS1S2_df_s1['area'], kr_singleS1S2_df_s1['rise_time'], s=0.1, color = 'blue', alpha = 0.5)
plt.scatter(kr_singleS1S2_df_s2['area'], kr_singleS1S2_df_s2['rise_time'], s=0.1, color = 'green', alpha = 0.5)
plt.scatter(kr_dS1sS2_df_s1['area'], kr_dS1sS2_df_s1['rise_time'], s=0.1, color = 'cyan', alpha = 0.5)
plt.scatter(kr_dS1sS2_df_s2['area'], kr_dS1sS2_df_s2['rise_time'], s=0.1, color = 'lime', alpha = 0.5)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
kr_s1_sS1S2 = []
len_sS1S2_s1 = len(kr_singleS1S2_df_s1)
len_dS1sS2_s1 = len(kr_dS1sS2_df_s1)
len_sS1S2_s2 = len(kr_singleS1S2_df_s2)
len_dS1sS2_s2 = len(kr_dS1sS2_df_s2)
for idx in kr_singleS1S2_df_s1.index:
    kr_s1_sS1S2.append(idx) #single S1
for idx2 in kr_dS1sS2_df_s1.index:
    kr_s1_sS1S2.append(idx2) #double S1
for idx3 in kr_singleS1S2_df_s2.index:
    kr_s1_sS1S2.append(idx3) #single S2 from single S1
for idx4 in kr_dS1sS2_df_s2.index:
    kr_s1_sS1S2.append(idx4) #single s2 from double S1

In [ ]:
s1s_num = len(kr_singleS1S2_df_s1) + len(kr_dS1sS2_df_s1)
S2s_num = len(kr_singleS1S2_df_s2) + len(kr_dS1sS2_df_s2)

In [ ]:
S2s_num

In [ ]:
#kr_s1_sS1S2

In [ ]:
s1s_num

In [ ]:
len(kr_singleS1S2_df_s1)

In [ ]:
len(kr_s1_sS1S2)

In [ ]:
len(np.unique(kr_singleS1S2_df_s1))

In [ ]:
len(kr_singleS1S2_df_s2)

In [ ]:
len(kr_dS1sS2_df_s2)

In [ ]:
#kr_s1_sS1S2 indecies

In [ ]:
cal_run_id = '034373'
kr_events_array2 = st.get_array(cal_run_id, targets =('events', 'cut_Kr_SingleS1S2', 'cut_Kr_DoubleS1_SingleS2'))
kr_pb_array2 = st.get_array(cal_run_id, targets =('peak_basics'))
window2 = strax.touching_windows(kr_pb_array2, kr_events_array2[kr_events_array2['cut_Kr_DoubleS1_SingleS2'] == True])
store_values2 = []
for i in np.arange(len(window2)):
    store_values2.append(np.arange(int(window2[i,0]), window2[i,1]+1))
    
values2 = np.concatenate(store_values2)
fig = plt.figure(figsize=(12,6))
plt.scatter(kr_pb_array2['area'][values2], kr_pb_array2['rise_time'][values2], s=0.1, color = 'blue', alpha = 0.5)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
window1 = strax.touching_windows(kr_pb_array, kr_events_array[kr_events_array['cut_Kr_DoubleS1_SingleS2'] == True])

In [ ]:
window2 = strax.touching_windows(kr_pb_array, kr_events_array[kr_events_array['cut_Kr_SingleS1S2'] == True])

In [ ]:
clean_peaks = peak_pro[peak_pro['n_competing_left']==0]

In [ ]:
clean_peaks

In [ ]:
s1 = kr_pb_array[kr_pb_array['type']==1]

In [ ]:
s2 = kr_pb_array[kr_pb_array['type']==2]

In [ ]:
s0 = kr_pb_array[kr_pb_array['type']==0]

In [ ]:
s1_peaks = kr_peaks[kr_peaks['type'] == 1]
s2_peaks = kr_peaks[kr_peaks['type'] == 2]

In [ ]:
save_s1=[] #give indecies in the S1 space as to the location of peaks
for j in clean_peaks:
    index = np.where(s1['time']==j['time']) 
    if len(index[0]) != 0:
        save_s1.append(index[0][0])

In [ ]:
save_s2=[]
for j in clean_peaks:
    index = np.where(s2['time']==j['time']) 
    if len(index[0]) != 0:
        save_s2.append(index[0][0])

In [ ]:
save_s=[]
for j in clean_peaks:
    index = np.where(s0['time']==j['time']) 
    if len(index[0]) != 0:
        save_s.append(index[0][0])

In [ ]:
s1=s1[save_s1]
s2=s2[save_s2]
#s0=s0[save_s]

In [ ]:
s1_peaks = s1_peaks[save_s1]
s2_peaks = s2_peaks[save_s2]

In [ ]:
np.argwhere(kr_pb_array['time'] == s1_peaks['time'])

In [ ]:
def make_index_list_with_time(data, s_peaks):
    total_num_indecies = len(s_peaks)
    data_indecies = np.zeros(total_num_indecies)
    for i in np.arange(total_num_indecies):
        data_indecies[i] = np.argwhere(data['time'] == s_peaks['time'][i])
        
    return data_indecies

In [ ]:
s1_index = make_index_list_with_time(kr_pb_array, s1)

In [ ]:
s2_index = make_index_list_with_time(kr_pb_array, s2)

In [ ]:
s1_index.astype(int)

In [ ]:
s2_index.astype(int)

In [ ]:
bg_data = np.concatenate((s1_peaks, s2_peaks))

In [ ]:
bg_data_index = np.concatenate((s1_index.astype(int), s2_index.astype(int)))

In [ ]:
data_indecies = np.concatenate((kr_s1_sS1S2, bg_data_index))

In [ ]:
_,ordered_index = np.unique(data_indecies, return_index=True)

In [ ]:
data_indecies_ordered = data_indecies[np.sort(ordered_index )]

In [ ]:
np.savez('./saved_data/data_indecies_ordered.npz', data_indecies_ordered)

In [ ]:
s1_peaks['time'][:10]

In [ ]:
#save_s1

In [ ]:
bg_data_pb = np.concatenate((s1,s2))

In [ ]:
n_complete_s1s2 = np.concatenate((save_s1,save_s2))

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(s1['area'], s1['rise_time'], s=0.1, color = 'blue')
plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig = plt.figure(figsize=(9,6))
plt.scatter(s1['area'], s1['rise_time'], s=0.1, color = 'blue')
plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
###Delete after done this is just a sanity check

In [ ]:
def assign_labels(data, ref_img, xdim, ydim, cut_out):
    '''This functions takes in the data and classifications based on an image gives the
    unique labels as well as the data set bacl with the new classification
    PS this version only takes in S1s and S2s and ignores unclassified samples, 
    another version will be made to deal with the unclassified samples
    
    data: can be either peaks or peak_basics
    ref_img: will be the image extracted from the SOM classification of each data point
    xdim: width of the image cube
    ydim: height of the image cube
    cut_out: removes the n last digits of the image vector if necesarry'''
    from PIL import Image
    img = Image.open(ref_img)
    imgGray = img.convert('L')
    #imgGray2.save('/home/luissanchez25/im_kr83_real__30x30_2lbl.0.rmpmap.png')
    img_color = np.array(img)
    img_color_2d = img_color.reshape((xdim*ydim,3))
    label = -1 * np.ones(img_color.shape[:-1])
    colorp = np.unique(img_color_2d, axis = 0)
    for i, color in enumerate(colorp):  # argwhere
        label[np.all((img_color == color), axis = 2)] = i
    label_vec = label.reshape((xdim*ydim))
    if cut_out != 0:
        label_vec_nonzero = label_vec[:-cut_out]
    elif cut_out == 0:
        label_vec_nonzero = label_vec
    s2_data = data[data['type'] == 2]
    s1_data = data[data['type'] == 1]
    som_class_peaks = np.concatenate((s2_data,s1_data)) #note had to reverse s2,s1 to s1,s2
    som_class_peaks['type'] = label_vec_nonzero
    
    return colorp, som_class_peaks

In [ ]:
color_check, som_cls_check = assign_labels(pb_data_test[data_index_test], './SOM_results/kr_clean.map.png', 152, 991, 1)

In [ ]:

fig = plt.figure(figsize=(15,9))
plt.scatter(som_cls_check['area'][som_cls_check['type'] == 1], som_cls_check['rise_time'][som_cls_check['type'] == 1], s=0.1, color = 'orange')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
#kr_s1_sS1S2
#Generate krypton peaks indecies
data_indecies = np.append(kr_s1_sS1S2, n_complete_s1s2.astype(int))

In [ ]:
data_indecies[:10]

In [ ]:
_,indx = np.unique(data_indecies, return_index=True)

In [ ]:
data_indecies = data_indecies[np.sort(indx)]

In [ ]:
np.max(data_indecies)

In [ ]:
len(kr_pb_array)

In [ ]:
kr_pb_array[data_indecies]

In [ ]:
#np.savez('./saved_data/data_indecies_info.npz', data_indecies)

In [ ]:
kr_s1_sS1S2[:10]

In [ ]:
n_complete_s1s2[:10]

In [ ]:
len_kr_plus_s1 = len(np.unique(np.concatenate((kr_pb_array[kr_s1_sS1S2],s1))))

In [ ]:
len(np.unique(np.concatenate((np.concatenate((kr_pb_array[kr_s1_sS1S2],s1)),s2))))

In [ ]:
len_kr_plus_s1

In [ ]:
len((np.concatenate((kr_pb_array,s1))))

In [ ]:
len_kr_plus_s1

In [ ]:
len(s1)

In [ ]:
1+1

In [ ]:
Data_4_SOM = np.concatenate((kr_peaks[kr_s1_sS1S2], bg_data))

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(Data_4_SOM['area'][0:len(kr_pb_array[kr_s1_sS1S2])], Data_4_SOM['rise_time'][0:len(kr_pb_array[kr_s1_sS1S2])], s=0.1, color = 'orange')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
np.save(kr_peaks[kr_s1_sS1S2], )

In [ ]:
#Data_4_SOM

In [ ]:
data_index_p = np.concatenate((kr_s1_sS1S2,save_s1))
data_index_kr = np.concatenate((data_index_p,save_s2))

In [ ]:
#Data_4_SOM

In [ ]:
sep_points = np.array([16327, 16327+len(kr_dS1sS2_df_s1), len(kr_s1_sS1S2),
                       (kr_plus_bgs1)])#single s1, double s1, kr s2, s1,s2
S2s_num

In [ ]:
sep_points2 = np.array([s1s_num, s1s_num+S2s_num, len_kr_plus_s1,
                        152*991])#single s1, double s1, kr s2, s1,s2

In [ ]:
sep_points2

In [ ]:
sep_points

In [ ]:
len_kr_plus_s1

In [ ]:
152*991

In [ ]:
kr_plus_bgs1 = len(np.unique(np.concatenate((kr_peaks[kr_s1_sS1S2], s1_peaks))))

In [ ]:
kr_plus_bgs1

In [ ]:
len(bg_data)

In [ ]:
len(kr_peaks[kr_s1_sS1S2])

In [ ]:
_,data_index = np.unique(Data_4_SOM , return_index=True)

In [ ]:
_, data_index_all = np.unique(data_index_kr , return_index=True)

In [ ]:
proxy_var = np.concatenate((np.array(kr_s1_sS1S2), np.array(save_s1)))

In [ ]:
filtered_data_indecies = np.concatenate((proxy_var, np.array(save_s2)))

In [ ]:
#sanity check
kr_pb_array_filtered = kr_pb_array[filtered_data_indecies]
kr_pb_som_data = kr_pb_array_filtered[np.sort(data_index_test)]

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(kr_pb_array['area'][bg_data_index], 
            kr_pb_array['rise_time'][bg_data_index], 
            s=0.1, color = 'blue')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
#data_indecies_ordered
fig = plt.figure(figsize=(15,9))
plt.scatter(kr_pb_array['area'][data_indecies_ordered][0:40445], 
            kr_pb_array['rise_time'][data_indecies_ordered][0:40445], 
            s=0.1, color = 'blue')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(kr_pb_som_data['area'][0:len(kr_pb_array[kr_s1_sS1S2])], kr_pb_som_data['rise_time'][0:len(kr_pb_array[kr_s1_sS1S2])], 
            s=0.1, color = 'blue')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
#np.savez("./saved_data/kr_clean_data_index_034373.npz",data_index_all)

In [ ]:
Data_4_SOM = Data_4_SOM[data_index] #check if it worked by doing the same with peak basics

In [ ]:
pb_data_test = np.concatenate((kr_pb_array[kr_s1_sS1S2], bg_data_pb))

In [ ]:
len(kr_pb_array[kr_s1_sS1S2])

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(pb_data_test['area'][0:len(kr_pb_array[kr_s1_sS1S2])], pb_data_test['rise_time'][0:len(kr_pb_array[kr_s1_sS1S2])], s=0.1, color = 'orange')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
#np.savez('./saved_data/clean_kr_cal_pb.npz', pb_data_test)

In [ ]:
len(kr_pb_array[kr_s1_sS1S2])

In [ ]:
len(bg_data_pb)

In [ ]:
_,data_index_test = np.unique(pb_data_test, return_index=True, unique_indices = True)

In [ ]:
pd_data_unique = pb_data_test[np.sort(data_index_test)]

In [ ]:
data_index_test[:10]

In [ ]:
def unique_data_org_order(data):
    data_rev = data[::-1]
    _,data_index_test = np.unique(data_rev, return_index=True)
    data_rev_ordered = data_rev[np.sort(data_index_test)]
    result = data_rev_ordered[::-1]
    return result

In [ ]:
a = [1, 5, 8, 7, 2]
a[::-1]

In [ ]:
pd_data_ordered = unique_data_org_order(pb_data_test)

In [ ]:
sdpoj = pb_data_test[np.sort(data_index_test)]

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(pb_data_test['area'][data_index_test], pb_data_test['rise_time'][data_index_test], s=0.1, color = 'blue')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(pd_data_ordered['area'][0:40445], pd_data_ordered['rise_time'][0:40445], s=0.1, color = 'blue')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(pd_data_unique['area'][0:40445], pd_data_unique['rise_time'][0:40445], s=0.1, color = 'blue')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'orange')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
pb_data_test[data_index_test]

In [ ]:
#np.savez('./saved_data/kr_clean_pb_data_v2.npz', pb_data_test[np.sort(data_index_test)])

In [ ]:
np.savez('./saved_data/filtered_ordered_indecies.npz', data_index_test.astype(int))

In [ ]:
len(pb_data_test[data_index_test])

In [ ]:
len(Data_4_SOM)

In [ ]:
#save data
np.savez('cleaned_kr_cal_data_peaks.npz', Data_4_SOM)

# Data has been processed and filtered! Now its time to converty it into the desired formats and export it along with the classification file!

In [ ]:
def compute_wf_and_quantiles(peaks: np.ndarray, bayes_n_nodes: int):
    """
    Compute waveforms and quantiles for a given number of nodes(atributes)
    :param peaks:
    :param bayes_n_nodes: number of nodes or atributes
    :return: waveforms and quantiles
    """
    waveforms = np.zeros((len(peaks), bayes_n_nodes))
    quantiles = np.zeros((len(peaks), bayes_n_nodes))

    num_samples = peaks['data'].shape[1] 
    #modified line, original num_samples = peaks['data'].shape[1] 
    step_size = int(num_samples/bayes_n_nodes)
    steps = np.arange(0, num_samples+1, step_size)

    data = peaks['data'].copy() #data = peaks['data'].copy() 
    data[data < 0.0] = 0.0
    for i, p in enumerate(peaks):
        sample_number = np.arange(0, num_samples+1, 1)*p['dt']
        frac_of_cumsum = np.append([0.0], np.cumsum(data[i, :]) / np.sum(data[i, :]))
        cumsum_steps = np.interp(np.linspace(0., 1., bayes_n_nodes, endpoint=False), frac_of_cumsum, sample_number)
        cumsum_steps = np.append(cumsum_steps, sample_number[-1])
        quantiles[i, :] = cumsum_steps[1:] - cumsum_steps[:-1]

    for j in range(bayes_n_nodes):
        waveforms[:, j] = np.sum(data[:, steps[j]:steps[j+1]], axis=1)
    waveforms = waveforms/(peaks['dt']*step_size)[:, np.newaxis]

    del data
    return waveforms, quantiles

In [ ]:
def L1_norm_c(data):
    #first 0 out all negative values
    data = np.where(data<0, 0, data)
    #now L1 normalize
    peaks_L1 = np.zeros((np.shape(data)))
    for aa in np.arange(len(data[:,1])):
        peaks_L1[aa,:] = data[aa,:]/np.sum(data[aa,:])     
        
    return peaks_L1

In [ ]:
def generate_incl_file_4(filename, xcube, ycube, class_nums):
    '''generates a file that tells NS what the label of each data point is. this needs:
    filename: such as 'data.incl'
    xcube: horizontal dimension of the data cube
    ycube: vertical dimension of the data cube
    class_nums: vector of 3 dimensions detailing where the labels should swtich, the values of each vector should be its own puls the previous'''
    f = open(filename, "w+")
    count = 1
    for i in np.arange(ycube):
        for j in np.arange(xcube):
            if count <= class_nums[0]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' A' + ' \n')
            if count > class_nums[0] and count <= class_nums[1]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' H' + '\n')
            if count > class_nums[1] and count <= class_nums[2]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' G' + '\n')
            if count > class_nums[2] and count <= class_nums[3]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' C' + '\n')
            if count > class_nums[3]:
                f.write('exclude area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1))
            count = count + 1
    f.close()

In [ ]:
def export_data(data, export_name):
    data_vec = np.reshape(data, (len(data[1,:])*len(data[:,1])))
    
    import struct

    f=open(export_name,"wb")
    export_TL = data_vec
    export_TL.dtype
    myfmt='f'*len(export_TL)
    #  You can use 'd' for double and < or > to force endinness
    bin=struct.pack(myfmt,*export_TL)
    f.write(bin)
    f.close()

In [ ]:
wf50, qaunt50 = compute_wf_and_quantiles(Data_4_SOM, 50)

In [ ]:
wf10, deciles = compute_wf_and_quantiles(Data_4_SOM, 10)

In [ ]:
np.concatenate((wf50,qaunt50))

In [ ]:
L1_norm_wf50 = L1_norm_c(wf50)

In [ ]:
np.min(L1_norm_wf50)

In [ ]:
#quant50 = np.log10(qaunt50) 

In [ ]:
np.max(qaunt50)

In [ ]:
quant50_scaled = (qaunt50 - np.min(qaunt50)) * (1- 0) / (np.max(qaunt50) - np.min(qaunt50)) + 0

In [ ]:
np.min(quant50_scaled)

In [ ]:
np.min(quant50_scaled)

In [ ]:
np.min(qaunt50)

In [ ]:
data_50quant_wf = np.concatenate((quant50_scaled, L1_norm_wf50), axis = 1) #all dave between 0 , 1

In [ ]:
np.shape(data_50quant_wf)

In [ ]:
plt.plot(data_50quant_wf[0,:])
#plt.yscale('log')

In [ ]:
plt.plot(data_50quant_wf[10,:])

In [ ]:
plt.plot(data_50quant_wf[100000,:])

In [ ]:
len(data_50quant_wf)

In [ ]:
def factors(n):    
    from functools import reduce
    
    return set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

In [ ]:
factors(150632) # need to add 1 vector with zeros

In [ ]:
152*991

In [ ]:
data_area = Data_4_SOM['area']
data_area = np.where(data_area > 1, data_area, 1)
np.min(data_area)

In [ ]:
np.min(np.log10(data_area))

In [ ]:
#deciles + log area
deciles_l1 = L1_norm_c(deciles)
data_area = Data_4_SOM['area']
data_area = np.where(data_area > 1, data_area, 1)
wf_area = data_area 
wf_area = np.log10(wf_area)
wf_area_scaled = (wf_area - np.min(wf_area)) * (1- 0) / (np.max(wf_area) - np.min(wf_area)) + 0
data_deciles_larea = np.concatenate((deciles_l1, np.reshape(wf_area_scaled, [len(wf_area_scaled),1])), axis = 1)

In [ ]:
np.min(Data_4_SOM['area'])

In [ ]:
len(deciles_l1)

In [ ]:
np.shape(deciles_l1)

In [ ]:
np.shape(wf_area_scaled)

In [ ]:
check = np.concatenate((deciles_l1,np.reshape(wf_area_scaled, [len(wf_area_scaled),1])), axis = 1)

In [ ]:
check

In [ ]:
plt.plot(check[200])

In [ ]:
np.shape(deciles_l1)

In [ ]:
np.argwhere(Data_4_SOM['area'] == np.min(Data_4_SOM['area']))

In [ ]:
data_area = Data_4_SOM

In [ ]:
np.min(data_area['area'])

In [ ]:
data_area['area'] = np.where(data_area['area'] > 0, data_area['area'], 0.0001) #assign arbitrary small value for negative area

In [ ]:
np.min(data_area['area'])

In [ ]:
np.max(data_area['area'])

In [ ]:
#deciles + log area
deciles_l1 = L1_norm_c(deciles)
wf_area = data_area['area']
#wf_area = np.log10(wf_area)
wf_area_scaled = (wf_area - np.min(wf_area)) * (1- 0) / (np.max(wf_area) - np.min(wf_area)) + 0
data_deciles_larea = np.concatenate((deciles_l1, wf_area_scaled), axis = 1)

In [ ]:
data_deciles_larea = np.concatenate((deciles_l1, wf_area_scaled), axis = 1)

In [ ]:
wf_area_scaled = np.reshape(wf_area_scaled, [len(wf_area_scaled),1])
np.shape(wf_area_scaled)

In [ ]:
data_deciles_larea = np.concatenate((deciles_l1, wf_area_scaled), axis = 1)

In [ ]:
data_deciles_larea.shape

In [ ]:
data_50quant_wf_zeros = np.concatenate((data_50quant_wf, np.reshape(np.zeros(100), [1,100])), axis = 0)

In [ ]:
plt.plot(data_50quant_wf_zeros[1,:])

In [ ]:
data_50quant_wf_zeros.shape

In [ ]:
data_50quant_wf.shape

In [ ]:
np.reshape(np.zeros(100), [1,100]).shape

In [ ]:
np.shape(data_deciles_larea)

In [ ]:
#data_deciles_larea
data_deciles_larea_zeros = np.concatenate((data_deciles_larea, np.reshape(np.zeros(11), [1,11])), axis = 0)

In [ ]:
export_data(data_50quant_wf_zeros, './saved_data/run_034373_50quant_scaled_50wfL1v2.raw')

In [ ]:
export_data(data_deciles_larea_zeros, './saved_data/run_034373_10decile_logareav2.raw')

In [ ]:
sep_points = np.array([16327, 16327+len(kr_dS1sS2_df_s1), len(kr_s1_sS1S2),
                       kr_plus_bgs1])#single s1, double s1, kr s2, s1,s2

In [ ]:
generate_incl_file_4('./saved_data/kr_clean_data_run_034373.incl',152, 991, sep_points)

In [ ]:
generate_incl_file_4('./saved_data/kr_clean_run_034373_v3.incl',152, 991, sep_points2)

In [ ]:
sep_points2

In [ ]:
sep_points = np.array([16327, 16327+len(kr_dS1sS2_df_s1), len(kr_s1_sS1S2),
                       kr_plus_bgs1])#single s1, double s1, kr s2, s1,s2

In [ ]:
sep_points2

In [ ]:
#Finally lets generate the include files and call it there

In [ ]:
np.array([1,2,3,4]).shape

In [ ]:
wf_area_scaled.shape

In [ ]:
deciles_l1.shape

In [ ]:
plt.plot(Data_4_SOM['data'][12462]) # -.- we have datapoints with negative area... could just set to 0

In [ ]:
np.shape(wf_area_scaled)

In [ ]:
wf_area_scaled

In [ ]:
len(wf_area_scaled)

In [ ]:
len(wf10)

In [ ]:
store_s1s2_time = np.zeros(len(s1) + len(s2))
store_s1s2_endtime = np.zeros(len(s1) + len(s2))

In [ ]:
store_s1s2_time[:len(s1)] = s1['time']
store_s1s2_time[len(s1):] = s2['time']

store_s1s2_endtime[:len(s1)] = s1['endtime']
store_s1s2_endtime[len(s1):] = s2['endtime']

In [ ]:
s1

In [ ]:
data_time_kr_plus_bg = np.zeros(len(store_s1s2_time)+len(start_time_kr))
data_endtime_kr_plus_bg = np.zeros(len(store_s1s2_time)+len(start_time_kr))

In [ ]:
#got the actual peaks so will use that instead
#kr_s1_sS1S2 #kr indecies
kr_peaks_clean = kr_peaks[kr_s1_sS1S2]
kr_peaks_bg = kr_peaks[kr_peaks['time'] == s1['time']]

In [ ]:
S1_loc = []
S2_loc = []
for w in np.arange(len(s1)):
    S1_loc.append(np.argwhere(kr_peaks['time'] == s1['time'][w]))
    
for ww in np.arange(len(s2)):
    S2_loc.append(np.argwhere(kr_peaks['time'] == s2['time'][ww]))
#np.argwhere(kr_peaks['time'] == s1['time'])
s1s2_loc = np.concatenate((S1_loc, S2_loc))

In [ ]:
s1s2_loc.shape

In [ ]:
s1s2_loc_nokr = [j for j in s1s2_loc if j not in kr_s1_sS1S2] #remove events allready in the kr set

In [ ]:
s1s2_loc = np.reshape(s1s2_loc, (len(s1s2_loc)))

In [ ]:
i = 0
reset_points = time_resets
while i in np.arange(len())

points_to_del = []
loop_check = 0
j=0
for i, idx in enumerate(s1s2_loc):
    if reset_points[loop_check] == i:
        i = 0
        loop_check = 1
        
    while j < len(kr_s1_sS1S2):
        points_to_del.np.argwhere
    n = len(peaklets_start)
    time_index = endtime_index = 0 #indecies of peaklets to check
    time_check = 0 #index to reset the time when needed
    

    
    for i, t0 in enumerate(peak_window_start):
        if time_resets[time_check] == i:
            time_index = endtime_index = 0 #reset the timer once in a while since that data is not all organizing according to time
            time_check += 1
            print("Reset number " + str(time_check))
            
        #gap_start = 0
        #gap_end = 0
        while peaklets_start[time_index] < t0:
            time_index += 1

In [ ]:
points_to_del = []
for b in np.arange(len(kr_s1_sS1S2)):
    points_to_del.append(np.argwhere(kr_s1_sS1S2[b] == s1s2_loc))

In [ ]:
point_to_del_ar = np.array(new_list)

In [ ]:
point_to_del_ar = np.reshape(point_to_del_ar, len(point_to_del_ar))

In [ ]:
np.in1d(s1s2_loc_nokr,kr_s1_sS1S2).any()

In [ ]:
s1s2_loc_nokr = np.delete(s1s2_loc, point_to_del_ar)

In [ ]:
len(np.concatenate((kr_s1_sS1S2, s1s2_loc_nokr), axis = 0))

In [ ]:
_,data_by_index = np.unique(np.concatenate((kr_s1_sS1S2, s1s2_loc_nokr), axis = 0), return_index=True)
data_by_index = np.sort()

In [ ]:
_,data_by_idx = np.unique(np.concatenate((kr_s1_sS1S2,s1s2_loc), axis = 0), return_index=True)
data_by_idx = np.sort(data_by_idx)
len(data_by_idx)

In [ ]:
kr_peaks_filtered = kr_peaks[data_by_idx]

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(s1['area'], s1['rise_time'], s=0.1, color = 'blue')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
kr_peaks_filtered.dtype

In [ ]:
kr_pb_filtered = kr_pb_array[data_by_idx]

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(kr_pb_array['area'][kr_s1_sS1S2],  kr_pb_array['rise_time'][kr_s1_sS1S2], s=0.1, color = 'blue')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(kr_pb_array['area'][n_complete_s1s2],  kr_pb_array['rise_time'][n_complete_s1s2], s=0.1, color = 'blue')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
kr_pc_s1_ncom = kr_pb_array[n_complete_s1s2]
kr_pc_s1_ncom = kr_pc_s1_ncom[kr_pc_s1_ncom['type'] == 1]

In [ ]:
kr_pc_s2_ncom = kr_pb_array[n_complete_s1s2]
kr_pc_s2_ncom = kr_pc_s2_ncom[kr_pc_s2_ncom['type'] == 2]

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(kr_pc_s1_ncom['area'],  kr_pc_s1_ncom['rise_time'], s=0.1, color = 'blue')
plt.scatter(kr_pc_s2_ncom['area'],  kr_pc_s2_ncom['rise_time'], s=0.1, color = 'green')
#plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig = plt.figure(figsize=(10,7))
plt.scatter(s1['area'], s1['rise_time'], s=0.1, color = 'blue')
plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.scatter(s1['area'], s1['rise_time'], s=0.1, color = 'blue')
plt.scatter(s2['area'], s2['rise_time'], s=0.1, color = 'green')
#plt.scatter(s0['area'], s0['rise_time'], s=0.1, color = 'black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')

In [ ]:
time_resets

In [ ]:
len(np.unique(kr_s1_sS1S2[21780]))

In [ ]:
len(np.unique(kr_s1_sS1S2))

In [ ]:
check1

In [ ]:
np.unique(s1s2_loc)

In [ ]:
np.concatenate((kr_s1_sS1S2,s1s2_loc), axis = 0)

In [ ]:
np.unique(np.concatenate((kr_s1_sS1S2,s1s2_loc), axis = 0), return_index=True)

In [ ]:
new_list = list(filter(None, points_to_del))

In [ ]:
points_to_del[points_to_del.astype(bool)]

In [ ]:
s1s2_loc_nokr

In [ ]:
1+1

In [ ]:
len(kr_s1_sS1S2)

In [ ]:
len(np.unique(kr_s1_sS1S2))

In [ ]:
len(kr_s1_sS1S2[s1s_num:])

In [ ]:
len(np.unique(kr_s1_sS1S2[s1s_num:]))

In [ ]:
kr_peaks_bg

In [ ]:
#get all the data together in 1 array
data_time_kr_plus_bg[:len(start_time_kr)] = start_time_kr
data_time_kr_plus_bg[len(start_time_kr):] = store_s1s2_time

data_endtime_kr_plus_bg[:len(start_time_kr)] = endtime_kr
data_endtime_kr_plus_bg[len(start_time_kr):] = store_s1s2_endtime

In [ ]:
#get all the unique values but keep them in order!!!
_,idx1 = np.unique(data_time_kr_plus_bg, return_index=True)
time_idx_sorted = np.sort(idx1)
data_time_clean = data_time_kr_plus_bg[time_idx_sorted]

In [ ]:
_,idx2 = np.unique(data_endtime_kr_plus_bg, return_index=True)
endtime_idx_sorted = np.sort(idx2)
data_endtime_clean = data_endtime_kr_plus_bg[endtime_idx_sorted]

In [ ]:
len(data_time_clean) == len(data_endtime_clean)

Now I have the time indecies I wanted for all the data, up next, selection the peaklets that fall in this category, will try to take inspiration from the touching windows function, maybe modify it for my purposes...

Cant do that since peaklets dont have an endtime attached to them...

In [ ]:
kr_peaklets_array.dtype

In [ ]:
peaklets_from_peaks=_touching_windows(kr_peaklets_array['time'], kr_peaklets_array['endtime'],
                                      data_time_clean, data_endtime_clean)

In [ ]:
time_resets = np.array([len(kr_singleS1S2_df_s1), len(kr_singleS1S2_df_s1)+len(kr_dS1sS2_df_s1),
                       len(kr_singleS1S2_df_s1)+len(kr_dS1sS2_df_s1)+len(kr_singleS1S2_df_s2),
                       len(kr_singleS1S2_df_s1)+len(kr_dS1sS2_df_s1)+len(kr_singleS1S2_df_s2)+len(kr_dS1sS2_df_s2),
                       10**9])

In [ ]:
peaklet_indecies = get_data_in_time_window(kr_peaklets_array['time'], data_time_clean, data_endtime_clean, time_resets)

In [ ]:
peaklet_indecies

In [ ]:
peaklet_indecies[time_resets[2]]

In [ ]:
len(kr_pb_array)

In [ ]:
len(kr_peaklets_array)

In [ ]:
plt.plot(kr_peaklets_array['data'][30])

In [ ]:
kr_peaklets_array['area'][30]

In [ ]:
plt.plot(kr_peaklets_array['data'][30])

In [ ]:
plt.plot(kr_peaklets_array['data'][31])

In [ ]:
plt.plot(kr_peaklets_array['data'][502])

In [ ]:
plt.plot(kr_peaklets_array['data'][503])

In [ ]:
#s2
plt.plot(kr_peaklets_array['data'][219245])

In [ ]:
#s2
plt.plot(kr_peaklets_array['data'][219246])

In [ ]:
len(peaklet_indecies)

In [ ]:
kr_peaklets_array['time'][-1] - data_endtime_clean[-1]

In [ ]:
len(kr_peaklets_array['time'])

In [ ]:
time_resets

In [ ]:
bg_run_id = '033859'

In [ ]:
bg_peaklets_array = st.get_array(bg_run_id, targets =('peaklets'))

In [ ]:
bg_pb_array = st.get_array(bg_run_id, targets =('peak_basics'))

In [ ]:
bg_peak_pro = st.get_array(bg_run_id,'peak_proximity')

In [ ]:
bg_peak_pro

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.scatter(bg_pb_array['area'][bg_pb_array['type'] == 2], bg_pb_array['rise_time'][bg_pb_array['type'] == 2], s=0.1, color = 'green', alpha = 0.5)
plt.scatter(bg_pb_array['area'][bg_pb_array['type'] == 1], bg_pb_array['rise_time'][bg_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
clean_bg_peaks = bg_peak_pro[bg_peak_pro['n_competing_left']==0]

In [ ]:
kr_pb_array.dtype

In [ ]:
def find_data_in_selection(selection, data, cut):
    
    select_data = np.argwhere(selection[cut] == True)
    #start_time = selection['time'][select_data]
    #end_time = selection['endtime'][select_data]
    store_indecies = np.zeros(len(data))
    
    count = 0
    for i in np.arange(len(select_data)):
        count2 = 0
        for j in np.arange(len(data)):
            if ((data['time'][j] >= selection['time'][select_data[i]]) & (data['endtime'][j] <= selection['endtime'][select_data[i]])) == True:
                #print(data['time'][j] >= selection['time'][select_data[i]])
                #print(data['endtime'][j] <= selection['endtime'][select_data[i]])
                #print(data['time'][j] >= selection['time'][select_data[i]] & data['endtime'][j] <= selection['endtime'][select_data[i]])
                print(j)
                store_indecies[count] = j
                count = count + 1
                count2 = count2 + 1
            if count2 == 4:
                break
    return store_indecies

In [ ]:
False & False

In [ ]:
kr_events_array.dtype

In [ ]:
kr_ss_events = find_data_in_selection(kr_events_array, kr_pb_array, 'cut_Kr_SingleS1S2')

In [ ]:
kr_events

In [ ]:
np.size(kr_pb_array['area'][kr_pb_array['type'] == 1])

In [ ]:
indecies = np.argwhere(kr_events_array['cut_Kr_SingleS1S2'] == True)
indecies[0]

In [ ]:
kr_events_array['endtime'][indecies[0]] - kr_events_array['time'][indecies[0]]

In [ ]:
kr_events_array['endtime'][4] - kr_events_array['time'][4]

In [ ]:
kr_events_array['time'][4] 

In [ ]:
len(kr_pb_array)

In [ ]:
kr_pb_array['time'][25] - kr_events_array['time'][4] 

In [ ]:
kr_pb_array['endtime'][28] - kr_events_array['endtime'][4] 

In [ ]:
kr_pb_array['time'][25] - kr_events_array['time'][8] 

In [ ]:
np.argwhere(kr_pb_array['time'] >= kr_events_array['time'][4] )

In [ ]:
kr_pb_array['time'][8]

In [ ]:
start = kr_events_array['time'][np.argwhere(kr_events_array['cut_Kr_SingleS1S2'] == True)]

In [ ]:
end = kr_events_array['endtime'][np.argwhere(kr_events_array['cut_Kr_SingleS1S2'] == True)]

In [ ]:
kr_pb_array['time'] >= start  & kr_pb_array['endtime'] <= end

In [ ]:
kr_pb_array[kr_pb_array['time'] >= start  & kr_pb_array['endtime'] <= end]

In [ ]:
kr_pb_df = pd.DataFrame(kr_pb_array)
kr_pb_df

In [ ]:
kr_events_array = st.get_array(cal_run_id, targets =('events', 'cut_Kr_SingleS1S2', 'cut_Kr_DoubleS1_SingleS2'))
kr_pb_array = st.get_array(cal_run_id, targets =('peak_basics'))
window1 = strax.touching_windows(kr_pb_df, kr_events_array[kr_events_array['cut_Kr_SingleS1S2'] == True])
store_values = []
for i in np.arange(len(window1)):
    store_values.append(np.arange(int(window1[i,0]), window1[i,1]+1))
    

In [ ]:
#store_values

In [ ]:
len(kr_peaklets_array)

In [ ]:
len(kr_pb_array)

In [ ]:
kr_peaklets_array.dtype

In [ ]:
plt.plot(np.mean(kr_peaklets_array['data'][:20], axis = 0), color = 'blue', alpha = 1)

So here we have a bit of an issue with indecies... non of the arrays are the same size so indexes are kinda useless. Instead we need to base things around the time of certain events. I can apply the peak_pro cut and change the labels of some of the krypton events (could take a step further and also label the argon events but ill avoid this for now).
I will try to use the time stamps of things, recompute the rise-time value in the peaklets

In [ ]:
def peaklet_data_selection(peaklets, desired_pb_data):
    '''
    This function attempts to do the following:
    '''

In [ ]:
def desired_peaks(pb_data, selection)

In [ ]:
def peaks_limits()

In [ ]:
def eliminate_peaklets()

In [ ]:
data = np.load('./saved_data/cleaned_kr_cal_data_peaks.npz')

In [ ]:
pwd

In [ ]:
np.shape(data)

In [ ]:
data

In [ ]:
np.shape(data['arr_0'][0])

In [ ]:
150631/200

In [ ]:
np.array(data['arr_0'][0])

In [ ]:
np.size(data['arr_0'][0])

In [ ]:
generate_incl_file_4('./saved_data/incl_krdata_bgdata.incl', 152,991, [21487,40145,74510, 160000])

In [ ]:
def generate_incl_file_4(filename, xcube, ycube, class_nums):
    '''generates a file that tells NS what the label of each data point is. this needs:
    filename: such as 'data.incl'
    xcube: horizontal dimension of the data cube
    ycube: vertical dimension of the data cube
    class_nums: vector of 3 dimensions detailing where the labels should swtich, the values of each vector should be its own puls the previous'''
    f = open(filename, "w+")
    count = 1
    for i in np.arange(ycube):
        for j in np.arange(xcube):
            if count <= class_nums[0]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' A' + ' \n')
            if count > class_nums[0] and count <= class_nums[1]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' H' + '\n')
            if count > class_nums[1] and count <= class_nums[2]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' G' + '\n')
            if count > class_nums[2] and count <= class_nums[3]:
                f.write('include area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' C' + '\n')
            if count > class_nums[3]:
                f.write('exclude area ' 
                        +str(j+1) + ' ' 
                        +str(i+1) + ' ' 
                        +str(j+1) + ' ' 
                        +str(i+1))
            count = count + 1
    f.close()

In [ ]:
#Here I will remake the data file (In case something I did was stupid and messed things up) with some modifications

In [ ]:
kr_peaks.dtype

In [ ]:
kr_pb_array.dtype

In [ ]:
kr_peak_data_for_training = kr_peaks[data_indecies_ordered]

In [ ]:
kr_pb_data_for_training = kr_pb_array[data_indecies_ordered]

In [ ]:
def compute_wf_and_quantiles(peaks: np.ndarray, bayes_n_nodes: int):
    """
    Compute waveforms and quantiles for a given number of nodes(atributes)
    :param peaks:
    :param bayes_n_nodes: number of nodes or atributes
    :return: waveforms and quantiles
    """
    waveforms = np.zeros((len(peaks), bayes_n_nodes))
    quantiles = np.zeros((len(peaks), bayes_n_nodes))

    num_samples = peaks['data'].shape[1] 
    #modified line, original num_samples = peaks['data'].shape[1] 
    step_size = int(num_samples/bayes_n_nodes)
    steps = np.arange(0, num_samples+1, step_size)

    data = peaks['data'].copy() #data = peaks['data'].copy() 
    data[data < 0.0] = 0.0
    for i, p in enumerate(peaks):
        sample_number = np.arange(0, num_samples+1, 1)*p['dt']
        frac_of_cumsum = np.append([0.0], np.cumsum(data[i, :]) / np.sum(data[i, :]))
        cumsum_steps = np.interp(np.linspace(0., 1., bayes_n_nodes, endpoint=False), frac_of_cumsum, sample_number)
        cumsum_steps = np.append(cumsum_steps, sample_number[-1])
        quantiles[i, :] = cumsum_steps[1:] - cumsum_steps[:-1]

    for j in range(bayes_n_nodes):
        waveforms[:, j] = np.sum(data[:, steps[j]:steps[j+1]], axis=1)
    waveforms = waveforms/(peaks['dt']*step_size)[:, np.newaxis]

    del data
    return waveforms, quantiles

In [ ]:
_, peak_data_SOM = compute_wf_and_quantiles(kr_peak_data_for_training, 10)

In [ ]:
np.shape(peak_data_SOM)

In [ ]:
plt.plot(np.mean(peak_data_SOM[kr_peak_data_for_training['type'] == 1], axis = 0), color = 'blue')
plt.plot(np.mean(peak_data_SOM[kr_peak_data_for_training['type'] == 2], axis = 0), color = 'green')
plt.yscale('log')

In [ ]:
log_area = np.log10(kr_peak_data_for_training['area'] + np.abs(np.min(kr_peak_data_for_training['area']))+1)

In [ ]:
np.savez('area_scaling.npz', np.abs(np.min(kr_peak_data_for_training['area'])))

In [ ]:
np.min(log_area)

In [ ]:
np.max(log_area)

In [ ]:
log_area_scaled = log_area / np.max(log_area) #area data

In [ ]:
np.savez('area_norm.npz', np.max(log_area))

In [ ]:
kr_pb_data_for_training['area_fraction_top']

In [ ]:
np.min(kr_pb_data_for_training['area_fraction_top'])

In [ ]:
np.max(kr_pb_data_for_training['area_fraction_top'])

In [ ]:
kr_pb_data_for_training = kr_pb_array[data_indecies_ordered] #this is just loading kr_peak_basics and making some data selection

In [ ]:
area_fraction_top = kr_pb_data_for_training['area_fraction_top']

In [ ]:
area_fraction_top[np.isnan(area_fraction_top)] = 0

In [ ]:
np.max(area_fraction_top)

In [ ]:
np.min(area_fraction_top)

In [ ]:
#area_fraction_top[area_fraction_top < 0] = 0

In [ ]:
np.argwhere(area_fraction_top < 0)

In [ ]:
plt.plot(kr_peak_data_for_training['data'][42908])
plt.title("Waveform of peak with negative AFT")
plt.xlabel('Time 10[ns]')
plt.ylabel('Raw Amplitude [PE]')

In [ ]:
kr_peak_data_for_training['type'][42908]

In [ ]:
kr_peak_data_for_training['dt'][42908]

In [ ]:
unphysical_aft_idx = np.argwhere(area_fraction_top > 1)

In [ ]:
unphysical_aft_idx.shape

In [ ]:
unphysical_aft_idx[1,0].astype(int)

In [ ]:
len(unphysical_aft_idx)

In [ ]:
plt.plot(kr_peak_data_for_training['data'][unphysical_aft_idx[0,0].astype(int)])

In [ ]:
plt.plot(kr_peak_data_for_training['data'][unphysical_aft_idx[8,0].astype(int)])

In [ ]:
for u in np.arange(len(unphysical_aft_idx)):
    plt.plot(kr_peak_data_for_training['data'][unphysical_aft_idx[u,0]], color = 'red', alpha = 0.3)
plt.title("Waveform of peak with AFT > 1")
plt.xlabel('Time 10[ns]')
plt.ylabel('Raw Amplitude [PE]')

In [ ]:
unphysical_aft_idx.reshape(16)

In [ ]:
for u in np.arange(len(unphysical_aft_idx)):
    plt.scatter(kr_peak_data_for_training['data'][unphysical_aft_idx[u,0]], color = 'red', alpha = 0.3)
plt.title("Waveform of peak with AFT > 1")
plt.xlabel('Time 10[ns]')
plt.ylabel('Raw Amplitude [PE]')

In [ ]:
kr_peak_data_for_training['dt'][unphysical_aft_idx]

In [ ]:
area_fraction_top_no_nan

In [ ]:
np.argwhere(area_fraction_top == np.min(area_fraction_top))

In [ ]:
area_fraction_top = np.where(area_fraction_top > 0, area_fraction_top, 0)

In [ ]:
area_fraction_top = np.where(area_fraction_top < 1, area_fraction_top, 1)

In [ ]:
np.max(area_fraction_top)

In [ ]:
area_fraction_top[area_fraction_top < 0.0]

In [ ]:
np.where(area_fraction_top > 0 , area_fraction_top, 

In [ ]:
np.argwhere(area_fraction_top == np.max(area_fraction_top))

In [ ]:
plt.plot(kr_peak_data_for_training['area_per_channel'][42908])
plt.xlabel("Channel number")
plt.ylabel("PE seen per channel")

In [ ]:
plt.plot(kr_peak_data_for_training['area_per_channel'][42908], color = 'blue', label = 'minimum AFT')
plt.plot(kr_peak_data_for_training['area_per_channel'][42488], color = 'red', label = 'maximum AFT')
plt.xlabel("Channel number")
plt.ylabel("PE seen per channel")
plt.legend()

In [ ]:
np.max(peak_data_SOM, axis = 0)

In [ ]:
np.min(peak_data_SOM, axis = 0)

In [ ]:
peak_data_log = np.log(peak_data_SOM)

In [ ]:
np.max(peak_data_log, axis = 0)

In [ ]:
np.max(log_area)

In [ ]:
peak_data_log_n = peak_data_log/ np.max(peak_data_log)

In [ ]:
np.max(peak_data_log)

In [ ]:
np.savez('deciles_scaling.npz', peak_data_log)

In [ ]:
log_area_n = log_area/ np.max(log_area)

In [ ]:
log_area_n = np.reshape(log_area_n, [len(log_area_n),1])

In [ ]:
decile_area = np.concatenate((peak_data_log_n, log_area_n), axis = 1)

In [ ]:
decile_area_aft = np.concatenate((decile_area, np.reshape(area_fraction_top, [len(area_fraction_top), 1])), axis = 1)

In [ ]:
log_area #shifted by absolute value of min + 1

In [ ]:
log_data_pt1 = np.concatenate((peak_data_log, np.reshape(log_area, [len(log_area),1])), axis = 1)

In [ ]:
log_data = np.concatenate((log_data_pt1, np.reshape(area_fraction_top, [len(area_fraction_top), 1])), axis = 1)

In [ ]:
import plotly.graph_objects as go
 
# initialize a Figure object and store it in
# a variable fig
fig = go.Figure()
 
# add x and y values for the 1st scatter
# plot and name the yaxis as yaxis1 values
fig.add_trace(go.Scatter(
    x=[10, 12, 13],
    y=[41, 58, 60],
    name="yaxis1 values"
))
 
# add x and y values for the 2nd scatter
# plot and name the yaxis as yaxis2 values
fig.add_trace(go.Scatter(
    x=[12, 13, 14],
    y=[401, 501, 610],
    name="yaxis2 values",
    yaxis="y2"
))
 
# add x and y values for the 3rd scatter
# plot and name the yaxis as yaxis3 values
fig.add_trace(go.Scatter(
    x=[14, 15, 16],
    y=[42000, 53000, 65000],
    name="yaxis3 values",
    yaxis="y3"
))
 
# add x and y values for the 4th scatter plot
# and name the yaxis as yaxis4 values
fig.add_trace(go.Scatter(
    x=[15, 16, 17],
    y=[2000, 5000, 7000],
    name="yaxis4 values",
    yaxis="y4"
))
 
# Create axis objects
fig.update_layout(
    # split the x-axis to fraction of plots in
    # proportions
    xaxis=dict(
        domain=[0.3, 0.7]
    ),
   
    # pass the y-axis title, titlefont, color
    # and tickfont as a dictionary and store
    # it an variable yaxis
    yaxis=dict(
        title="yaxis 1",
        titlefont=dict(
            color="#0000ff"
        ),
        tickfont=dict(
            color="#0000ff"
        )
    ),
     
    # pass the y-axis 2 title, titlefont, color and
    # tickfont as a dictionary and store it an
    # variable yaxis 2
    yaxis2=dict(
        title="yaxis 2",
        titlefont=dict(
            color="#FF0000"
        ),
        tickfont=dict(
            color="#FF0000"
        ),
        anchor="free",  # specifying x - axis has to be the fixed
        overlaying="y",  # specifyinfg y - axis has to be separated
        side="left",  # specifying the side the axis should be present
        position=0.2  # specifying the position of the axis
    ),
   
    # pass the y-axis 3 title, titlefont, color and
    # tickfont as a dictionary and store it an
    # variable yaxis 3
    yaxis3=dict(
        title="yaxis 3",
        titlefont=dict(
            color="#006400"
        ),
        tickfont=dict(
            color="#006400"
        ),
        anchor="x",     # specifying x - axis has to be the fixed
        overlaying="y",  # specifyinfg y - axis has to be separated
        side="right"  # specifying the side the axis should be present
    ),
     
    # pass the y-axis 4 title, titlefont, color and
    # tickfont as a dictionary and store it an
    # variable yaxis 4
    yaxis4=dict(
        title="yaxis 4",
        titlefont=dict(
            color="#8f00ff"
        ),
        tickfont=dict(
            color="#8f00ff"
        ),
        anchor="free",  # specifying x - axis has to be the fixed
        overlaying="y",  # specifyinfg y - axis has to be separated
        side="right",  # specifying the side the axis should be present
        position=0.8  # specifying the position of the axis
    )
)
 
 
# Update layout of the plot namely title_text, width
# and place it in the center using title_x parameter
# as shown
fig.update_layout(
    title_text="4 y-axes scatter plot in plotly",
    width=1000,
    title_x=0.5
)

In [ ]:
from plotly.subplots import make_subplots
x = np.arange(12)
x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
fig = plt.figure(1)
ax = fig.add_subplot(111)
#fig = make_subplots(specs=[[{"secondary_y": True}]])
ax.plot(np.mean(decile_area_aft[kr_peak_data_for_training['type'] == 1], axis = 0), color = 'blue', label = "Straxen S1")
ax.plot(np.mean(decile_area_aft[kr_peak_data_for_training['type'] == 2], axis = 0), color = 'green', label = "Straxen S2")

#fig.update_yaxes(title_text= "Maximum Normalized Log10 Deciles + Area", secondary_y=False)
#fig.update_yaxes(title_text= "Area Fraction Top", secondary_y=True)

ax.set_xticks(x)
ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)
ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)")
ax2 = ax.twinx()
#ax2.bar(x, y1, alpha=0.5) 
ax2.set_ylabel('Area Fraction Top')
#ax2.set_xticks(x)
#ax2.set_xticklabels([])

ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)", fontsize=10)
ax.set_title("SOM mean input vectors for Straxen classified S1s and S2s")
ax.legend()
#plt.yscale('log')

In [ ]:
from plotly.subplots import make_subplots
x = np.arange(12)
x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
fig = plt.figure(1)
ax = fig.add_subplot(111)
#fig = make_subplots(specs=[[{"secondary_y": True}]])

for q in np.arange(200):
    ax.plot(decile_area_aft[kr_peak_data_for_training['type'] == 1][q], color = 'blue', alpha = 0.1, label = "Straxen S1")
    ax.plot(decile_area_aft[kr_peak_data_for_training['type'] == 2][q], color = 'green', alpha = 0.1,label = "Straxen S2")

#fig.update_yaxes(title_text= "Maximum Normalized Log10 Deciles + Area", secondary_y=False)
#fig.update_yaxes(title_text= "Area Fraction Top", secondary_y=True)

ax.set_xticks(x)
ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)
ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)")
ax2 = ax.twinx()
#ax2.bar(x, y1, alpha=0.5) 
ax2.set_ylabel('Area Fraction Top')
#ax2.set_xticks(x)
#ax2.set_xticklabels([])

ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)", fontsize=10)
ax.set_title("500 examples of input vectors form Straxen classified S1s and S2s")
#ax.legend()
#plt.yscale('log')

In [ ]:
from plotly.subplots import make_subplots
x = np.arange(12)
x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
fig = plt.figure(1)
ax = fig.add_subplot(111)
#fig = make_subplots(specs=[[{"secondary_y": True}]])

for q in np.arange(500):
    ax.plot(decile_area_aft[kr_peak_data_for_training['type'] == 1][q], color = 'blue', alpha = 0.05, label = "Straxen S1")
    ax.plot(decile_area_aft[kr_peak_data_for_training['type'] == 2][q], color = 'green', alpha = 0.05,label = "Straxen S2")

#fig.update_yaxes(title_text= "Maximum Normalized Log10 Deciles + Area", secondary_y=False)
#fig.update_yaxes(title_text= "Area Fraction Top", secondary_y=True)

ax.set_xticks(x)
ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)
ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)")
ax2 = ax.twinx()
#ax2.bar(x, y1, alpha=0.5) 
ax2.set_ylabel('Area Fraction Top')
#ax2.set_xticks(x)
#ax2.set_xticklabels([])

ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)", fontsize=10)
#ax.set_title("500 examples of input vectors form Straxen classified S1s and S2s")
plt.savefig('./figures/500_examples_of_inputs.png')
#ax.legend()
#plt.yscale('log')

In [ ]:
from plotly.subplots import make_subplots
x = np.arange(12)
x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
fig = plt.figure(1)
ax = fig.add_subplot(111)
#fig = make_subplots(specs=[[{"secondary_y": True}]])

for q in np.arange(500):
    ax.plot(decile_area_aft[kr_peak_data_for_training['type'] == 1][q], color = 'blue', alpha = 0.05, label = "Straxen S1")
    ax.plot(decile_area_aft[kr_peak_data_for_training['type'] == 2][q], color = 'green', alpha = 0.05,label = "Straxen S2")

#fig.update_yaxes(title_text= "Maximum Normalized Log10 Deciles + Area", secondary_y=False)
#fig.update_yaxes(title_text= "Area Fraction Top", secondary_y=True)

ax.set_xticks(x)
ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)
ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)")
ax2 = ax.twinx()
#ax2.bar(x, y1, alpha=0.5) 
ax2.set_ylabel('Area Fraction Top')
#ax2.set_xticks(x)
#ax2.set_xticklabels([])

ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)", fontsize=10)
#ax.legend()
#ax.set_title("500 examples of input vectors form Straxen classified S1s and S2s")
plt.savefig('./figures/500_examples_of_inputs.png')
#ax.legend()
#plt.yscale('log')

In [ ]:
decile_area_aft[kr_peak_data_for_training['type'] == 1][0]

In [ ]:
fig = plt.figure(1, figsize=(12, 12))

extent = 1, 12, 0, 3
im2 = plt.imshow(decile_area_aft[kr_peak_data_for_training['type'] == 1], 
                 cmap=plt.cm.Blues, alpha=.9, interpolation='bilinear',
                extent = extent)

In [ ]:
bin_s1_data = decile_area_aft[kr_peak_data_for_training['type'] == 1]

In [ ]:
np.shape(bin_s1_data)

In [ ]:
bin_s1_data = decile_area_aft[kr_peak_data_for_training['type'] == 1]
num_xbins = 12
num_ybins = 10
bin_storage = np.zeros((num_ybins, num_xbins))
for n in np.arange(num_xbins):
    for m in np.arange(num_ybins):
        bin_storage[m,n] = sum((m/10 < bin_s1_data[:,n])*(bin_s1_data[:,n] <= (m+1)/10))

In [ ]:
def bin_data(data, xbins, ybins):
    bin_s1_data = data
    num_xbins = xbins
    num_ybins = ybins
    bin_storage = np.zeros((num_ybins, num_xbins))
    for n in np.arange(num_xbins):
        for m in np.arange(num_ybins):
            bin_storage[m,n] = sum((m/ybins < bin_s1_data[:,n])*(bin_s1_data[:,n] <= (m+1)/ybins))
            
    return bin_storage

In [ ]:
extent = 1, 12, 0, 10

im2 = plt.imshow(bin_storage, 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='nearest',
                extent = extent)



In [ ]:
extent = 1, 12, 0, 10
im2 = plt.imshow(np.flip(bin_storage, 0), 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='nearest',
                extent = extent)

In [ ]:
s1_bins = bin_data(decile_area_aft[kr_peak_data_for_training['type'] == 1], 12, 25)

In [ ]:
s2_bins = bin_data(decile_area_aft[kr_peak_data_for_training['type'] == 2], 12, 25)

In [ ]:
extent = 1, 30, 0, 25
#x = np.arange(12) +0.5
#x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
#                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
#                 'area', 'aft' ]
fig = plt.figure(1)
ax = fig.add_subplot(111)
im2 = plt.imshow(np.flip(s1_bins, 0), 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='nearest',
                 extent = extent)

#ax.set_xticks(x)
#ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)
plt.colorbar()

In [ ]:
extent = 1, 30, 0, 25
#x = np.arange(12) +0.5
#x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
#                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
#                 'area', 'aft' ]
fig = plt.figure(1)
ax = fig.add_subplot(111)
im2 = plt.imshow(np.flip(np.log10(s1_bins+1), 0), 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='nearest',
                 extent = extent)

#ax.set_xticks(x)
#ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)
plt.colorbar()

In [ ]:
x

In [ ]:
extent = 1, 30, 0, 25
im2 = plt.imshow(np.flip(s2_bins, 0), 
                 cmap=plt.cm.Greens, alpha=.5, interpolation='nearest',
                extent = extent)
plt.colorbar()

In [ ]:
extent = 1, 30, 0, 25
im2 = plt.imshow(np.flip(np.log10(s2_bins+1), 0), 
                 cmap=plt.cm.Greens, alpha=.5, interpolation='nearest',
                extent = extent)
plt.colorbar()

In [ ]:
extent = 1, 30, 0, 25
im2 = plt.imshow(np.flip(np.log10(s2_bins+1), 0), 
                 cmap=plt.cm.Greens, alpha=.5, interpolation='bilinear',
                extent = extent)
plt.colorbar()

In [ ]:
x = np.arange(12)
x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
extent = 1, 12, 0, 10
im2 = plt.imshow(np.flip(bin_storage, 0), 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='nearest',
                extent = extent)
im2 = plt.imshow(np.flip(s2_bins, 0), 
                 cmap=plt.cm.Reds, alpha=.5, interpolation='nearest',
                extent = extent)

plt.xticks(x)
plt.xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)

In [ ]:
fig = plt.figure(1)
ax = fig.add_subplot(111)

x = np.arange(12)
x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
extent = 1, 12, 0, 10
im2 = ax.imshow(np.flip(bin_storage, 0), 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='nearest',
                extent = extent, label = "S1")
im2 = ax.imshow(np.flip(s2_bins, 0), 
                 cmap=plt.cm.Reds, alpha=.5, interpolation='nearest',
                extent = extent, label = "S2")

ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)")
ax2 = ax.twinx()
#ax2.bar(x, y1, alpha=0.5) 
ax2.set_ylabel('Area Fraction Top')
ax.set_xticks(x)
ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)
#ax.legend()


In [ ]:


fig = plt.figure(1)
ax = fig.add_subplot(111)

x = np.arange(12)
x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
extent = 1, 12, 0, 10
#ax.plot(np.mean(decile_area_aft[kr_peak_data_for_training['type'] == 1], axis = 0), color = 'blue', label = "Straxen S1")
#ax.plot(np.mean(decile_area_aft[kr_peak_data_for_training['type'] == 2], axis = 0), color = 'green', label = "Straxen S2")
im2 = ax.imshow(np.flip(bin_storage, 0), 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='nearest',
                extent = extent, label = "S1")
im2 = ax.imshow(np.flip(s2_bins, 0), 
                 cmap=plt.cm.Reds, alpha=.5, interpolation='nearest',
                extent = extent, label = "S2")


ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)")
ax2 = ax.twinx()
#ax2.bar(x, y1, alpha=0.5) 
ax2.set_ylabel('Area Fraction Top')
ax.set_xticks(x)
ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)
#ax.legend()
#plt.show()

In [ ]:
s1_bins2 = bin_data(decile_area_aft[kr_peak_data_for_training['type'] == 1], 12, 25)
s2_bins2 = bin_data(decile_area_aft[kr_peak_data_for_training['type'] == 2], 12, 25)

In [ ]:
fig = plt.figure(1, figsize = (6,6))
ax = fig.add_subplot(111)

x = np.arange(12)
#y = np.arange(11)/10
x_ticks_labels = ['0', '1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
extent = 1, 12, 0, 10
#ax.plot(np.mean(decile_area_aft[kr_peak_data_for_training['type'] == 1], axis = 0), color = 'blue', label = "Straxen S1")
#ax.plot(np.mean(decile_area_aft[kr_peak_data_for_training['type'] == 2], axis = 0), color = 'green', label = "Straxen S2")
im2 = ax.imshow(np.flip(s1_bins2, 0), 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='nearest',
                extent = extent, label = "S1")
im2 = ax.imshow(np.flip(s2_bins2, 0), 
                 cmap=plt.cm.Reds, alpha=.5, interpolation='nearest',
                extent = extent, label = "S2")


ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)")
#ax2 = ax.twinx()
#ax2.bar(x, y1, alpha=0.5) 
#ax2.set_ylabel('Area Fraction Top')
ax.set_xticks(x)
#ax.set_yticks(y)
ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)
ax.set_xlim(1)
#ax.set_ylim(0,3)
#ax2.set_ylim(0.3,0.7)
#ax.legend()
#plt.show()

In [ ]:
fig = plt.figure(1, figsize = (12,12))
ax = fig.add_subplot(111)

x = np.arange(12)
x_ticks_labels = ['0', '1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
extent = 1, 12, 0, 10
#ax.plot(np.mean(decile_area_aft[kr_peak_data_for_training['type'] == 1], axis = 0), color = 'blue', label = "Straxen S1")
#ax.plot(np.mean(decile_area_aft[kr_peak_data_for_training['type'] == 2], axis = 0), color = 'green', label = "Straxen S2")
im2 = ax.imshow(np.flip(s1_bins2, 0), 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='nearest',
                extent = extent, label = "S1")
im2 = ax.imshow(np.flip(s2_bins2, 0), 
                 cmap=plt.cm.Reds, alpha=.5, interpolation='nearest',
                extent = extent, label = "S2")


ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)")
#ax2 = ax.twinx()
#ax2.bar(x, y1, alpha=0.5) 
#ax2.set_ylabel('Area Fraction Top')
ax.set_xticks(x)
ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)
ax.set_xlim(1)
#ax.set_ylim(0,3)
#ax2.set_ylim(0.3,0.7)
#ax.legend()
#plt.show()

In [ ]:
fig = plt.figure(1)
ax = fig.add_subplot(111)

x = np.arange(12)
x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
extent = 1, 12, 0, 10
#ax.plot(np.mean(decile_area_aft[kr_peak_data_for_training['type'] == 1], axis = 0), color = 'blue', label = "Straxen S1")
#ax.plot(np.mean(decile_area_aft[kr_peak_data_for_training['type'] == 2], axis = 0), color = 'green', label = "Straxen S2")
im2 = ax.imshow(np.flip(bin_storage, 0), 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='bilinear',
                extent = extent, label = "S1")
im2 = ax.imshow(np.flip(s2_bins, 0), 
                 cmap=plt.cm.Reds, alpha=.5, interpolation='bilinear',
                extent = extent, label = "S2")


ax.set_ylabel("Maximum Normalized Log10(Deciles, Area)")
ax2 = ax.twinx()
#ax2.bar(x, y1, alpha=0.5) 
ax2.set_ylabel('Area Fraction Top')
ax.set_xticks(x)
ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=10)

In [ ]:
extent = 1, 12, 0, 10
im2 = plt.imshow(np.flip(bin_storage, 0), 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='bilinear',
                extent = extent)
im2 = plt.imshow(np.flip(s2_bins, 0), 
                 cmap=plt.cm.Reds, alpha=.5, interpolation='bilinear',
                extent = extent)

In [ ]:
bin_storage.shape

In [ ]:
sum((0.0 < bin_s1_data[:,1])*(bin_s1_data[:,1] <= 0.1))

In [ ]:
sum(0.1 < (bin_s1_data[:,1] <= 0.2))

In [ ]:
sum(bin_s1_data[:,1] <= 0.1)

In [ ]:
fig = plt.figure(1, figsize=(12, 12))

extent = 1, 12, 0, 3
im2 = plt.imshow(decile_area_aft[kr_peak_data_for_training['type'] == 2], 
                 cmap=plt.cm.Reds, alpha=.9, interpolation='bilinear',
                extent = extent)

In [ ]:
fig = plt.figure(1, figsize=(12, 12))

extent = 1, 12, 0, 3
im2 = plt.imshow(decile_area_aft[kr_peak_data_for_training['type'] == 2], 
                 cmap=plt.cm.Reds, alpha=.5, interpolation='bilinear',
                extent = extent)
im2 = plt.imshow(decile_area_aft[kr_peak_data_for_training['type'] == 1], 
                 cmap=plt.cm.Blues, alpha=.5, interpolation='bilinear',
                extent = extent)

In [ ]:
x = np.arange(12)
x_ticks_labels = ['1st decile','2nd decile','3rd decile','4th decile', '5th decile',
                 '6th decile', '7th decile','8th decile','9th decile', '10th decile',
                 'area', 'aft' ]
fig, ax = plt.subplots(1,1) 

plt.plot(np.mean(log_data[kr_peak_data_for_training['type'] == 1], axis = 0), color = 'blue')
plt.plot(np.mean(log_data[kr_peak_data_for_training['type'] == 2], axis = 0), color = 'green')

ax.set_xticks(x)
ax.set_xticklabels(x_ticks_labels, rotation='vertical', fontsize=18)
log_data

In [ ]:
np.shape(peak_data_log/ np.max(peak_data_log))

In [ ]:
np.shape(log_area/ np.max(log_area))

In [ ]:
np.shape(decile_area_aft)

In [ ]:
decile_area_aft_zero = np.concatenate((decile_area_aft, np.reshape(np.zeros(12), [1,12])), axis = 0)

In [ ]:
plt.plot(decile_area_aft_zero[0])

In [ ]:
decile_area_aft_zero[:3]

In [ ]:
np.shape(decile_area_aft_zero)

In [ ]:
export_data(decile_area_aft_zero, './saved_data/run_034373_10decile_logarea_aft.raw')

In [ ]:
np.mean(decile_area_aft_zero)

In [ ]:
np.std(decile_area_aft_zero)

In [ ]:
np.var(decile_area_aft_zero)